## An Illustration of long division algorithm by Donald E.Knuth in his book TAOCP

![](long-division-Donald-Knuth.png)

In [144]:

def to_dec(limbs, b=10):
    return sum([x*(b**i) for i, x in enumerate(limbs)])

def from_dec(d, b=10):
    limbs = []
    while d >=b:
        limbs.append(d%b)
        d = d // b
        
    limbs.append(d)
    
    return limbs

def short_mul(limbs, d, b=10):
    assert d < b, "d < b"
    out = []
    carry = 0
    for i, l in enumerate(limbs):
        x = l*d + carry
        out.append(x%b)
        carry = x // b
        # print(i, l, d, x, out[-1], carry)
        
    if carry > 0:
        out.append(carry)
    
    return out

def short_div(limbs, d, b=10):
    quotient = []
    carry = 0
    for i, l in enumerate(limbs[::-1]):
        x = l+carry*b
        q = x // d
        carry = x % d
        # print(f"l = {l}, x = {x}, q = {q}, carry = {carry}")
        if i > 0 or q > 0:
            quotient.insert(0, q)
        
    return quotient, carry

def normalize(limbs):
    """
    Little-endian limbs (least-significant limb first).
    Removes most-significant zero limbs (i.e., zeros at the end of the list),
    leaving at least one limb.
    """
    if not limbs:
        return [0]
    i = len(limbs) - 1
    while i > 0 and limbs[i] == 0:
        i -= 1
    return limbs[: i + 1]    

def cmp(a, b):
    return (a > b) - (a < b)

def limbs_cmp(x, y):
    if len(x) != len(y):
        return cmp(len(x), len(y))
    
    for xi, yi in zip(x[::-1],y[::-1]):
        ret = cmp(xi, yi)
        if ret != 0:
            return ret
    
    return 0
        
    
def limbs_sub(x, y, b=10, with_normalize=True):
    if len(y) < len(x):
        y = y + [0*(len(x)-len(y))]
        #print(y)
    
    z = []
    carry = 0
    for i, (xi, yi) in enumerate(zip(x, y)):
        yi = yi + carry
        if xi >= yi:
            zi = xi - yi
            carry = 0
        else:
            zi = xi + b - yi
            carry = 1
        z.append(zi)
    
    if with_normalize:
        z = normalize(z)
    
    return z
        


In [198]:
        
def long_div(dividend, divisor, b=10):
    assert divisor[-1] > 0, "divisor[-1] > 0"
    assert len(dividend) >= len(divisor), "len(dividend )>= len(divisor)"
    assert to_dec(dividend) >= to_dec(divisor), f"to_dec(dividend)({to_dec(dividend)})({dividend}) >= to_dec(divisor)({to_dec(divisor)}) ({divisor})"
    n = len(divisor)
    m = len(dividend) - n # The length of dividend is supposed to be m+n
    #print(f"dividend={dividend}, divisor={divisor}, m={m}, n={n}")
    
    dnorm = b // (divisor[-1] + 1) # b//(v[n-1] + 1)
    dividend = short_mul(dividend, dnorm)
    divisor = short_mul(divisor, dnorm)
    if len(dividend) < m+n+1:
        dividend.append(0)
    #print(f"dnorm={dnorm}, dividend={dividend}, len={len(dividend)}, divisor={divisor}")
    
    j = m
    quotient = []
    for j in range(m, -1, -1):
        #print("="*20)
        dv = (dividend[j+n] * b + dividend[j+n-1])
        #print(f"j={j}, dividend[j+n]={dividend[j+n]}, dividend[j+n-1]={dividend[j+n-1]}, dv={dv}, divisor[n-1]={divisor[n-1]}")
        
        q = dv // divisor[n-1]
        r = dv % divisor[n-1]
        #print(f"q={q}, r={r}, q*divisor[n-2]={q*divisor[n-2]}, b*r+dividend[j+n-2]={b*r+dividend[j+n-2]}")

        while q >= b or q*divisor[n-2] > (b*r+dividend[j+n-2]):
            q -= 1
            r += divisor[n-1]
            #print(f"[adjust 1] q={q}, r={r}")
            
            if r >= b:
                break
                
        u = dividend[j:j+n+1]
        v = divisor + [0]
        #print(f"u={u}, v={v}")
        while True:
            qv = short_mul(v, q)
            #print(f"v={v}, q={q}, qv={qv}")
            if limbs_cmp(u, qv) == -1:  #negative
                q -= 1
                continue
            else:
                u_left = limbs_sub(u, qv, with_normalize=False)
                dividend[j:j+n+1] = u_left
                #print(f"u_left={u_left}, dividend={dividend}")
                break
                
        if j == 0:
            remainder = u_left
                
        quotient.insert(0, q)
    
    remainder = normalize(remainder)
    remainder, r_remainder = short_div(remainder, dnorm)
    #print(f"remainder={remainder}, r_remainder={r_remainder}")
    assert r_remainder == 0, "r_remainder == 0"
    remainder = normalize(remainder)
    quotient = normalize(quotient)
    
    return quotient, remainder
        
        
    
        

In [37]:
import numpy as np

for _ in range(10):
    u = np.random.randint(10000, )
    v = np.random.randint(10)
    r1 = to_dec(short_mul(from_dec(u), v))
    print(u, v, r1, u*v)

544 8 4352 4352
8106 3 24318 24318
5143 9 46287 46287
262 3 786 786
7914 4 31656 31656
7923 7 55461 55461
7395 4 29580 29580
5417 5 27085 27085
2964 7 20748 20748
5115 1 5115 5115


In [32]:
import numpy as np

In [61]:
limbs_sub(from_dec(10807), from_dec(5723))

[3, 2, 7, 5, 0]


[4, 8, 0, 5]

In [67]:
for _ in range(10):
    y = np.random.randint(1000)
    x = y + np.random.randint(10000)
    z = to_dec(limbs_sub(from_dec(x), from_dec(y)))
    print(x, y, z, x-y)

8098 516 7582 7582
4232 459 3773 3773
1281 567 714 714
9238 854 8384 8384
6135 493 5642 5642
3772 716 3056 3056
9242 737 8505 8505
4774 436 4338 4338
2653 374 2279 2279
10502 654 9848 9848


In [120]:
for _ in range(10):
    y = np.random.randint(1, 10)
    x = y*np.random.randint(10000)
    z = to_dec(short_div(from_dec(x), y)[0])
    print(x, y, z, x//y)

14205 5 2841 2841
39716 4 9929 9929
40610 5 8122 8122
53280 8 6660 6660
23572 4 5893 5893
28416 3 9472 9472
4670 1 4670 4670
9824 4 2456 2456
4170 2 2085 2085
1740 2 870 870


In [199]:
long_div(from_dec(59350), from_dec(881))

([7, 6], [3, 2, 3])

In [203]:
for _ in range(1000):
    y = np.random.randint(10000, 100000000)
    x = np.random.randint(3, 1000)*y + np.random.randint(y)
    q, r = long_div(from_dec(x), from_dec(y))
    q_int = x//y
    r_int = x%y
    print(x, y, q_int, r_int, to_dec(q), to_dec(r))
    if q_int != to_dec(q) or r_int != to_dec(r):
        print("Error!")

29879519289 90567767 329 82723946 329 82723946
33560194002 97961059 342 57511824 342 57511824
6579971470 19239276 342 139078 342 139078
51454901832 68209167 754 25189914 754 25189914
10200901407 75165617 135 53543112 135 53543112
41633456029 82379349 505 31884784 505 31884784
75310811841 75991289 991 3444442 991 3444442
4480983955 16598697 269 15934462 269 15934462
418024679 449637 929 311906 929 311906
63921646664 87593894 729 65697938 729 65697938
32270408072 59985001 537 58462535 537 58462535
14911598669 99181309 150 34402319 150 34402319
37714590265 91470058 412 28926369 412 28926369
69430564131 88082764 788 21346099 788 21346099
5106965530 13353539 382 5913632 382 5913632
2156341834 5272144 409 34938 409 34938
5285292624 7800053 677 4656743 677 4656743
22767595248 24795927 918 4934262 918 4934262
970438790 3594810 269 3434900 269 3434900
61609026042 68619409 897 57416169 897 57416169
31754189889 69472830 457 5106579 457 5106579
5495389318 13308674 412 12215630 412 12215630
3866593

47779147155 49935624 956 40690611 956 40690611
41424199578 73801040 561 21816138 561 21816138
33055950720 42394966 779 30272206 779 30272206
21236887519 39762159 534 3894613 534 3894613
16909348900 91288888 185 20904620 185 20904620
13766668979 57115465 241 1841914 241 1841914
1933979039 36825672 52 19044095 52 19044095
32376376390 39351544 822 29407222 822 29407222
27241255686 69387796 392 41239654 392 41239654
18243918008 19637537 929 646135 929 646135
21589805171 24331094 887 8124793 887 8124793
6697922574 13091373 511 8230971 511 8230971
8119770631 9093587 892 8291027 892 8291027
37142282896 80551609 461 7991147 461 7991147
24120873605 51906065 464 36459445 464 36459445
18549269636 91307826 203 13780958 203 13780958
6335990829 29593241 214 3037255 214 3037255
32991083774 50314052 655 35379714 655 35379714
63233712888 64624517 978 30935262 978 30935262
1194043751 2548899 468 1159019 468 1159019
14801721848 41591981 355 36568593 355 36568593
3793699193 55873109 67 50200890 67 5020089

38291044225 85396541 448 33393857 448 33393857
6368039387 14203024 448 5084635 448 5084635
18380810327 32319770 568 23180967 568 23180967
72346649359 98562351 734 1883725 734 1883725
25117073338 75201049 333 75124021 333 75124021
10498308673 31568710 332 17496953 332 17496953
35621324947 61238384 581 41823843 581 41823843
4796611503 48803399 98 13878401 98 13878401
18669744788 34096111 547 19172071 547 19172071
3468589265 55448464 62 30784497 62 30784497
6488192913 23909664 271 8673969 271 8673969
29301663586 32976493 888 18537802 888 18537802
3379882233 19118310 176 15059673 176 15059673
41040710791 85165614 481 76050457 481 76050457
34886262233 61924134 563 22974791 563 22974791
20704241307 32050999 645 31346952 645 31346952
992093915 3619663 274 306253 274 306253
59123271352 88188746 670 36811532 670 36811532
31720489485 91705373 345 82135800 345 82135800
38219126517 52025158 734 32660545 734 32660545
49636608351 90636968 547 58186855 547 58186855
15431844553 17940175 860 3294053 86

In [208]:
B = 1<<64
u = to_dec([2,3,4,5], b=B)
v = to_dec([7,8], b=B)
print(u, v)
q = u // v
r = u % v
print(q, v)
print(from_dec(q, b=B), from_dec(r, b=B))

31385508676933403820540076583722085934420615884268374065154 147573952589676412935
212676479325586539663744438516399996928 147573952589676412935
[17582052945254416384, 11529215046068469759] [6052837899185946626, 3]


In [211]:
u1 = to_dec([17582052945254416384, 11529215046068469759], b=B) * to_dec([7,8], b=B) + \
  to_dec([6052837899185946626, 3], b=B)
from_dec(u1, b=B)

[2, 3, 4, 5]

In [216]:
a = to_dec([0xFFFF_FFFF_FFFF_FFFF, 0x0000_0000_0000_0002], b=B)
b = 3
a // b, a % b

(18446744073709551615, 2)

[18446744073709551615, 2]